In [1]:
# 查看个人持久化工作区文件
!ls /home/mw/project/

ruc_Class25Q1_community.csv	  ruc_Class25Q1_xtest.csv
ruc_Class25Q1_merged.csv	  ruc_Class25Q1_xtrain.csv
ruc_Class25Q1_predictiontest.csv  ruc_Class25Q1_ytest.csv
ruc_Class25Q1_train1.csv	  ruc_Class25Q1_ytrain.csv


In [2]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

quant4533


In [2]:
# 所有使用到的库
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request
import numpy as np
from sklearn.impute import KNNImputer
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 期中建模  
## 1. 数据预处理  
## 2. 特征工程  

这里你可以编写代码，文档  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


In [4]:
total = pd.read_csv('/home/mw/input/quant4533/ruc_Class25Q1_train.csv')  
print(total.head())

   城市    区域     板块    环线     小区名称        价格      房屋户型        所在楼层     建筑面积  \
0   0  79.0  111.0  二至三环    人定湖西里   6564200  2室1厅1厨1卫   中楼层 (共5层)    52.3㎡   
1   0  43.0  231.0  五至六环    龙跃苑四区   4174000  3室1厅1厨1卫    顶层 (共6层)  127.44㎡   
2   0  97.0   54.0  五至六环      名都园  16310000  4室2厅1厨4卫    底层 (共3层)  228.54㎡   
3   0  62.0  568.0  三至四环   保利海德公园   2834600     2房间2卫  低楼层 (共10层)    43.6㎡   
4   0  62.0  226.0  三至四环  京投银泰琨御府   1954000     1房间1卫  中楼层 (共10层)   39.85㎡   

     套内面积  ... 产权所属 抵押信息       房屋优势                        核心卖点  \
0     NaN  ...  非共有  NaN   装修、房本满五年        此房是南北通透小板楼，户型方正，格局合理   
1  123.7㎡  ...  非共有  NaN   装修、房本满五年              南北通透商品房自住装修无个税   
2     NaN  ...  非共有  NaN      房本满五年               有挑空客厅私家车库花园和*   
3  29.39㎡  ...  非共有  NaN  地铁、房本满五年、                         NaN   
4  29.94㎡  ...  非共有  NaN   地铁、房本满五年  此房朝向是南向户型，小区位于地铁6号线和10号线的*   

                                                户型介绍  \
0                     房子是南北通透户型方正采光好，前后没有遮挡视野好，通风效果好   
1  房子三居一卫，户型方正，南

In [5]:
total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84133 entries, 0 to 84132
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      84133 non-null  int64  
 1   区域      84133 non-null  float64
 2   板块      84133 non-null  float64
 3   环线      42726 non-null  object 
 4   小区名称    84133 non-null  object 
 5   价格      84133 non-null  int64  
 6   房屋户型    83528 non-null  object 
 7   所在楼层    84133 non-null  object 
 8   建筑面积    84133 non-null  object 
 9   套内面积    25146 non-null  object 
 10  房屋朝向    84133 non-null  object 
 11  建筑结构    83528 non-null  object 
 12  装修情况    83528 non-null  object 
 13  梯户比例    82438 non-null  object 
 14  配备电梯    75818 non-null  object 
 15  别墅类型    749 non-null    object 
 16  交易时间    84133 non-null  object 
 17  交易权属    84133 non-null  object 
 18  上次交易    55180 non-null  object 
 19  房屋用途    84131 non-null  object 
 20  房屋年限    54351 non-null  object 
 21  产权所属    84133 non-null  object 
 22

In [6]:
'''
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 示例数据

# 只选取 **套内面积已知** 的数据
total_known = total[total['套内面积'].notnull()]

# 画散点图
plt.figure(figsize=(8, 6))
sns.scatterplot(data=total_known, x='建筑面积', y='套内面积', color='blue', alpha=0.7)

# 画一条回归趋势线（拟合线）
sns.regplot(data=total_known, x='建筑面积', y='套内面积', scatter=False, color='red')

# 添加标题和标签
plt.xlabel('建筑面积 (平方米)')
plt.ylabel('套内面积 (平方米)')
plt.title('套内面积 vs. 建筑面积')

# 显示图表
plt.show()
'''

"\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# 示例数据\n\n# 只选取 **套内面积已知** 的数据\ntotal_known = total[total['套内面积'].notnull()]\n\n# 画散点图\nplt.figure(figsize=(8, 6))\nsns.scatterplot(data=total_known, x='建筑面积', y='套内面积', color='blue', alpha=0.7)\n\n# 画一条回归趋势线（拟合线）\nsns.regplot(data=total_known, x='建筑面积', y='套内面积', scatter=False, color='red')\n\n# 添加标题和标签\nplt.xlabel('建筑面积 (平方米)')\nplt.ylabel('套内面积 (平方米)')\nplt.title('套内面积 vs. 建筑面积')\n\n# 显示图表\nplt.show()\n"

In [7]:
total["环线"].value_counts()

二至三环     8994
内环内      8976
五至六环     5573
三至四环     4453
四至五环     2785
三环外      2197
外环外      2038
六环外      1927
一至二环     1864
内环至外环    1044
中环至外环     923
内环至中环     892
二环内       603
一环内       347
四环外       110
Name: 环线, dtype: int64

In [8]:
total["交易权属"].value_counts()

商品房        77866
已购公房        2511
拆迁还建房        798
二类经济适用房      579
经济适用房        462
一类经济适用房      397
动迁安置房        341
限价商品房        263
私产           231
集资房          227
央产房          155
房改房          135
定向安置房         55
使用权           51
售后公房          37
自住型商品房        25
Name: 交易权属, dtype: int64

In [9]:
total["产权所属"].value_counts()

非共有    70207
共有     13926
Name: 产权所属, dtype: int64

In [10]:
total["房屋优势"].value_counts()

装修              5810
地铁、房本满五年        5212
装修、房本满五年        4850
地铁              4199
地铁、装修、房本满五年     4146
、房本满五年          3700
地铁、             3281
装修、             3053
装修、房本满五年、       2873
装修、房本满两年、       2696
房本满五年           2683
地铁、装修、房本满五年、    2454
装修、房本满两年        2202
地铁、房本满两年        2073
地铁、装修           1991
、               1947
地铁、房本满五年、       1835
、房本满五年、         1771
、房本满两年          1742
地铁、装修、房本满两年     1719
地铁、装修、房本满两年、    1670
地铁、装修、          1503
、房本满两年、         1492
房本满两年           1485
地铁、房本满两年、       1018
房本满五年、           384
房本满两年、           280
Name: 房屋优势, dtype: int64

In [11]:
total["房屋年限"].value_counts()

满五年     29723
满两年     16120
未满两年     8508
Name: 房屋年限, dtype: int64

In [12]:
total["房屋用途"].value_counts() 

普通住宅      79853
商业办公类      1178
别墅         1064
商住两用       1025
车库          605
公寓          160
写字楼          76
底商           49
商业           45
酒店式公寓        42
公寓/公寓        13
公寓/住宅         7
新式里弄          4
四合院           3
公寓（住宅）        2
老公寓           2
住宅式公寓         1
平房            1
花园洋房          1
Name: 房屋用途, dtype: int64

In [13]:
total["别墅类型"].value_counts()

联排    455
叠拼    175
独栋     76
双拼     43
Name: 别墅类型, dtype: int64

In [14]:
total["装修情况"].value_counts()

精装    30529
其他    26186
简装    17343
毛坯     9470
Name: 装修情况, dtype: int64

In [15]:
total["房屋户型"].value_counts()

2室1厅1厨1卫    22688
3室2厅1厨2卫    11542
2室2厅1厨1卫     8913
1室1厅1厨1卫     8263
3室2厅1厨1卫     5906
            ...  
1室2厅0厨1卫        1
9室1厅1厨9卫        1
8室5厅1厨6卫        1
7室4厅2厨4卫        1
9室3厅1厨3卫        1
Name: 房屋户型, Length: 346, dtype: int64

In [16]:
total["所在楼层"].value_counts()

高楼层 (共6层)     3494
中楼层 (共6层)     3131
中楼层 (共7层)     2606
中楼层 (共8层)     2370
高楼层 (共33层)    2322
              ... 
底层 (共30层)        1
顶层 (共35层)        1
地下室 (共4层)        1
地下室 (共21层)       1
低楼层 (共74层)       1
Name: 所在楼层, Length: 260, dtype: int64

In [17]:
total["房屋朝向"].value_counts()

南 北         27537
南           24863
东南           9327
东            4321
西南           2815
            ...  
东 西 东南          1
东南 东北 西北        1
东 南 东南          1
北 东 东北          1
西南 南 西          1
Name: 房屋朝向, Length: 161, dtype: int64

In [18]:
total.describe()

,城市,区域,板块,价格,抵押信息,lon,lat,年份
count,84133.000000,84133.000000,84133.000000,8.413300e+04,0.0,84133.000000,84133.000000,84133.000000
mean,2.894845,54.400794,434.420263,1.971953e+06,NaN,113.368467,35.247384,2021.070507
std,2.146294,27.870968,225.487506,2.639962e+06,NaN,6.714156,5.582976,0.882858
min,0.000000,2.000000,0.000000,7.828000e+04,NaN,106.197420,29.258156,2015.000000
25%,1.000000,35.000000,244.000000,6.974000e+05,NaN,106.581097,29.675878,2021.000000
50%,2.000000,59.000000,435.000000,1.146500e+06,NaN,114.154249,34.240172,2021.000000
75%,5.000000,74.000000,628.000000,2.176000e+06,NaN,116.636774,39.925985,2022.000000
max,6.000000,102.000000,810.000000,7.995000e+07,NaN,127.169979,45.920575,2022.000000


In [19]:
'''
import matplotlib.pyplot as plt
import seaborn as sns

target_col = '价格'
features = [col for col in total.columns if col != target_col]

# 设置画布大小和布局
plt.figure(figsize=(20, 60))  # 调整画布大小以适应所有图形

# 循环绘制每个特征与目标变量的关系
for i, feature in enumerate(features, 1):
    plt.subplot(11, 3, i)  # 11行3列布局，共33个位置(31个图+空白)
    
    # 根据数据类型选择适当的图形
    if total[feature].dtype.name == 'category' or total[feature].dtype == 'object':
        # 分类变量使用箱线图
        sns.boxplot(data=total, x=total[feature].astype(str), y=target_col)
        plt.xticks(rotation=45)
    else:
        # 数值变量使用散点图
        sns.scatterplot(data=total, x=feature, y=target_col, alpha=0.5)
    
    plt.title(f'{target_col} vs {feature}')
    plt.tight_layout()

plt.show()
'''

"\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\ntarget_col = '价格'\nfeatures = [col for col in total.columns if col != target_col]\n\n# 设置画布大小和布局\nplt.figure(figsize=(20, 60))  # 调整画布大小以适应所有图形\n\n# 循环绘制每个特征与目标变量的关系\nfor i, feature in enumerate(features, 1):\n    plt.subplot(11, 3, i)  # 11行3列布局，共33个位置(31个图+空白)\n    \n    # 根据数据类型选择适当的图形\n    if total[feature].dtype.name == 'category' or total[feature].dtype == 'object':\n        # 分类变量使用箱线图\n        sns.boxplot(data=total, x=total[feature].astype(str), y=target_col)\n        plt.xticks(rotation=45)\n    else:\n        # 数值变量使用散点图\n        sns.scatterplot(data=total, x=feature, y=target_col, alpha=0.5)\n    \n    plt.title(f'{target_col} vs {feature}')\n    plt.tight_layout()\n\nplt.show()\n"

In [20]:
'''
import matplotlib.pyplot as plt
import seaborn as sns

target_col = '价格'
features = [col for col in total.columns if col != target_col]
features_per_page = 6  # 每页显示6个图

for page in range(0, len(features), features_per_page):
    current_features = features[page:page+features_per_page]
    
    plt.figure(figsize=(20, 12))
    for i, feature in enumerate(current_features, 1):
        plt.subplot(2, 3, i)  # 2行3列布局
        
        if total[feature].dtype.name in ['category', 'object']:
            sns.boxplot(data=total, x=total[feature].astype(str), y=target_col)
            plt.xticks(rotation=45)
        else:
            sns.scatterplot(data=total, x=feature, y=target_col, alpha=0.5)
        
        plt.title(f'{target_col} vs {feature}')
    
    plt.tight_layout()
    plt.show()
    '''

"\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\ntarget_col = '价格'\nfeatures = [col for col in total.columns if col != target_col]\nfeatures_per_page = 6  # 每页显示6个图\n\nfor page in range(0, len(features), features_per_page):\n    current_features = features[page:page+features_per_page]\n    \n    plt.figure(figsize=(20, 12))\n    for i, feature in enumerate(current_features, 1):\n        plt.subplot(2, 3, i)  # 2行3列布局\n        \n        if total[feature].dtype.name in ['category', 'object']:\n            sns.boxplot(data=total, x=total[feature].astype(str), y=target_col)\n            plt.xticks(rotation=45)\n        else:\n            sns.scatterplot(data=total, x=feature, y=target_col, alpha=0.5)\n        \n        plt.title(f'{target_col} vs {feature}')\n    \n    plt.tight_layout()\n    plt.show()\n    "

In [21]:
'''
# 确保所有数据都转换为字符串类型后去除平方米单位
total['建筑面积'] = total['建筑面积'].astype(str).str.replace('㎡', '', regex=True).astype(float)
total['套内面积'] = total['套内面积'].astype(str).str.replace('㎡', '', regex=True)
total['套内面积'] = pd.to_numeric(total['套内面积'], errors='coerce')  # 转换为数值，无法转换的变为 NaN

# 按建筑面积降序排序
total = total.sort_values(by='建筑面积', ascending=False).reset_index(drop=True)

# 使用建筑面积作为参考，但只填充套内面积
temp_total = total[['建筑面积', '套内面积']]
imputer = KNNImputer(n_neighbors=10)
total['套内面积'] = imputer.fit_transform(temp_total)[:, 1]  # 仅填充套内面积

# 确保填充后的套内面积不会超过建筑面积
total['套内面积'] = np.minimum(total['套内面积'], total['建筑面积'])

# 输出填充后的DataFrame
print(total)
'''

"\n# 确保所有数据都转换为字符串类型后去除平方米单位\ntotal['建筑面积'] = total['建筑面积'].astype(str).str.replace('㎡', '', regex=True).astype(float)\ntotal['套内面积'] = total['套内面积'].astype(str).str.replace('㎡', '', regex=True)\ntotal['套内面积'] = pd.to_numeric(total['套内面积'], errors='coerce')  # 转换为数值，无法转换的变为 NaN\n\n# 按建筑面积降序排序\ntotal = total.sort_values(by='建筑面积', ascending=False).reset_index(drop=True)\n\n# 使用建筑面积作为参考，但只填充套内面积\ntemp_total = total[['建筑面积', '套内面积']]\nimputer = KNNImputer(n_neighbors=10)\ntotal['套内面积'] = imputer.fit_transform(temp_total)[:, 1]  # 仅填充套内面积\n\n# 确保填充后的套内面积不会超过建筑面积\ntotal['套内面积'] = np.minimum(total['套内面积'], total['建筑面积'])\n\n# 输出填充后的DataFrame\nprint(total)\n"

### 对类别变量进行数值类型转换  
***注意：目标变量”房价“是连续变量   
已有数据编码可以统一编，填充缺失值要在划分后  
1. 环线：因为类别较多，且目标变量为连续变量，所以使用target coding，用类别对应的均值替换类别本身。要注意！：必须在训练集上计算均值，在测试集应用，否则会导致data leakage issue  
2. 小区名称：待定，需结合detail数据  
3. 房屋户型：拆分成4个独立数值特征，让模型可以学习每个部分（卧室、厅、厨、卫）对房价的不同影响。  
4. 所在楼层：在购房时，相对楼层比绝对楼层更重要，因此采用顺序编码或者（k折）目标编码（待定）  
5. 建筑面积and套内面积：去掉平方米变数值型。二者离散型大，先用KNN近邻填充，再求得房率，只用得房率一个变量  
6. 房屋朝向：将161个category分类为东西南北四个主朝向，再进行独热编码  
7. 建筑结构：用频数编码  
8. 装修情况：标签编码，有顺序关系  
9. 梯户比例：转换成数值型  
10. 配备电梯: 独热编码 0&1  
11. 别墅类型：独热编码  
12. 交易时间  
13. 交易权属：独热编码  
14. 上次交易  
15. 房屋用途：频数编码，对稀有类别有一定鲁棒性  
16. 房屋年限：标签编码，类别少，有顺序关系  
17. 产权所属：独热编码  0&1  
18. 房屋优势：简单：去掉 复杂：分四个独立列，独热编码  
19. 核心卖点：简易：0&1独热编码 复杂：jieba分词，词频统计，评分  
20. 户型介绍  
21. 周边配套   
22. 交通出行  
23. lon&lat：简易：直接做特征 复杂：地理划分，区域聚类***

### 3. 房屋户型

In [22]:
# 定义提取函数
def extract_numbers(layout):
    if not isinstance(layout, str):  # 处理 NaN 或 None
        layout = ""

    # 使用正则表达式提取数字
    match = re.findall(r'(\d+)室|(\d+)厅|(\d+)厨|(\d+)卫', layout)
    
    # 合并所有找到的数字
    numbers = [int(num) if num else 0 for group in match for num in group if num]

    # 确保返回的列表长度为 4
    while len(numbers) < 4:
        numbers.append(0)

    return numbers

# 使用 apply() 和 pd.Series 强制确保每行数据有 4 个数值
total[['卧室', '客厅', '厨房', '卫生间']] = total['房屋户型'].apply(lambda x: pd.Series(extract_numbers(x), index=['卧室', '客厅', '厨房', '卫生间']))

# 删除原始户型列
total.drop(columns=['房屋户型'], inplace=True)

# 打印结果
print(total)

       城市    区域     板块    环线     小区名称        价格        所在楼层     建筑面积    套内面积  \
0       0  79.0  111.0  二至三环    人定湖西里   6564200   中楼层 (共5层)    52.3㎡     NaN   
1       0  43.0  231.0  五至六环    龙跃苑四区   4174000    顶层 (共6层)  127.44㎡  123.7㎡   
2       0  97.0   54.0  五至六环      名都园  16310000    底层 (共3层)  228.54㎡     NaN   
3       0  62.0  568.0  三至四环   保利海德公园   2834600  低楼层 (共10层)    43.6㎡  29.39㎡   
4       0  62.0  226.0  三至四环  京投银泰琨御府   1954000  中楼层 (共10层)   39.85㎡  29.94㎡   
...    ..   ...    ...   ...      ...       ...         ...      ...     ...   
84128   6  71.0  176.0   NaN     翡翠明珠   1059500  高楼层 (共33层)   94.12㎡  79.36㎡   
84129   6  93.0  247.0  二至三环    明德8英里   1087900  中楼层 (共30层)  105.45㎡  83.23㎡   
84130   6  46.0  597.0   NaN     魏玛公馆    967200  高楼层 (共32层)    53.8㎡  39.42㎡   
84131   6  93.0  247.0  二至三环     万国花园   1095000   中楼层 (共7层)  136.74㎡     NaN   
84132   6  46.0  597.0   NaN   毛纺厂家属院    434700   中楼层 (共5层)   63.02㎡  55.72㎡   

      房屋朝向  ...                        

### 4. 所在楼层  



In [23]:
# 提取括号前的文本并替换原数据
total['所在楼层'] = total['所在楼层'].str.extract(r'(.+?)（')
# 使用 pd.get_dummies进行独热编码
total = pd.get_dummies(total, columns=['所在楼层'])
# 输出结果
print(total)

       城市    区域     板块    环线     小区名称        价格     建筑面积    套内面积 房屋朝向  建筑结构  \
0       0  79.0  111.0  二至三环    人定湖西里   6564200    52.3㎡     NaN  南 北  混合结构   
1       0  43.0  231.0  五至六环    龙跃苑四区   4174000  127.44㎡  123.7㎡  南 北  混合结构   
2       0  97.0   54.0  五至六环      名都园  16310000  228.54㎡     NaN    南  混合结构   
3       0  62.0  568.0  三至四环   保利海德公园   2834600    43.6㎡  29.39㎡    北  钢混结构   
4       0  62.0  226.0  三至四环  京投银泰琨御府   1954000   39.85㎡  29.94㎡    南  钢混结构   
...    ..   ...    ...   ...      ...       ...      ...     ...  ...   ...   
84128   6  71.0  176.0   NaN     翡翠明珠   1059500   94.12㎡  79.36㎡    东  框架结构   
84129   6  93.0  247.0  二至三环    明德8英里   1087900  105.45㎡  83.23㎡   东南  钢混结构   
84130   6  46.0  597.0   NaN     魏玛公馆    967200    53.8㎡  39.42㎡    北  钢混结构   
84131   6  93.0  247.0  二至三环     万国花园   1095000  136.74㎡     NaN  南 北  混合结构   
84132   6  46.0  597.0   NaN   毛纺厂家属院    434700   63.02㎡  55.72㎡    南  混合结构   

       ...                                         

### 5. 得房率  


In [24]:
'''
# 计算得房率（套内面积 / 建筑面积）
total['得房率'] = total['套内面积'] / total['建筑面积']
print(total[["得房率"]])
'''

'\n# 计算得房率（套内面积 / 建筑面积）\ntotal[\'得房率\'] = total[\'套内面积\'] / total[\'建筑面积\']\nprint(total[["得房率"]])\n'

### 6. 房屋朝向

In [25]:
# 按主要采光面方向分类（用每个数据的第一个字替换整个字符串）
total['房屋朝向'] = total['房屋朝向'].astype(str).str[0]
total['房屋朝向'] = total['房屋朝向'].fillna('其他')
# 创建一个映射字典
order_mapping = {
    '南': 0,
    '北': 1,
    '西': 2,
    '东': 3
}

# 使用映射字典进行编码
total['房屋朝向'] = total['房屋朝向'].map(order_mapping)

print(total[["房屋朝向"]])

       房屋朝向
0         0
1         0
2         0
3         1
4         0
...     ...
84128     3
84129     3
84130     1
84131     0
84132     0

[84133 rows x 1 columns]


### 7. 建筑结构

In [26]:
# 进行频数编码
total['建筑结构'] = total['建筑结构'].map(total['建筑结构'].value_counts())
# 查看结果
print(total[['建筑结构']].head())

      建筑结构
0   9405.0
1   9405.0
2   9405.0
3  65198.0
4  65198.0


### 8. 装修情况

In [27]:
total['装修情况'] = total['装修情况'].fillna('其他')
# 创建一个映射字典
order_mapping = {
    '其他': 0,
    '毛坯': 1,
    '简装': 2,
    '精装': 3
}

# 使用映射字典进行编码
total['装修情况'] = total['装修情况'].map(order_mapping)

print(total[["装修情况"]])

       装修情况
0         3
1         3
2         3
3         3
4         3
...     ...
84128     3
84129     3
84130     3
84131     3
84132     0

[84133 rows x 1 columns]


### 9. 梯户比例

In [28]:
# 中文数字转换为阿拉伯数字的映射
num_map = {'一': 1, '二': 2, '两': 2, '三': 3, '四': 4, '五': 5, '六': 6, 
           '七': 7, '八': 8, '九': 9, '十': 10}

# 处理中文数字（包括大于10的情况）
def chinese_to_number(chinese_num):
    if not chinese_num:  # 处理空值
        return None
    if chinese_num in num_map:
        return num_map[chinese_num]  # 直接映射
    elif '十' in chinese_num:
        parts = chinese_num.split('十')
        if parts[0] == '':  # "十X" 形式，如"十五"
            return 10 + num_map.get(parts[1], 0)
        elif parts[1] == '':  # "X十" 形式，如"二十"
            return num_map[parts[0]] * 10
        else:  # "X十Y" 形式，如"二十五"
            return num_map[parts[0]] * 10 + num_map.get(parts[1], 0)
    return None  # 无法识别的情况返回 None

# 解析 "X梯Y户" 并计算梯户比
def convert_ratio(text):
    text = str(text)  # 确保 text 为字符串
    match = re.findall(r'([\u4e00-\u9fa5]+)梯([\u4e00-\u9fa5]+)户', text)
    if match:
        ladders = chinese_to_number(match[0][0])  # 提取电梯数
        households = chinese_to_number(match[0][1])  # 提取户数
        if ladders and households:
            return round(ladders / households, 4)  # 计算梯户比
    return None  # 无法解析的情况

# 应用转换
total['梯户比例'] = total['梯户比例'].apply(convert_ratio)
print(total)


       城市    区域     板块    环线     小区名称        价格     建筑面积    套内面积  房屋朝向  \
0       0  79.0  111.0  二至三环    人定湖西里   6564200    52.3㎡     NaN     0   
1       0  43.0  231.0  五至六环    龙跃苑四区   4174000  127.44㎡  123.7㎡     0   
2       0  97.0   54.0  五至六环      名都园  16310000  228.54㎡     NaN     0   
3       0  62.0  568.0  三至四环   保利海德公园   2834600    43.6㎡  29.39㎡     1   
4       0  62.0  226.0  三至四环  京投银泰琨御府   1954000   39.85㎡  29.94㎡     0   
...    ..   ...    ...   ...      ...       ...      ...     ...   ...   
84128   6  71.0  176.0   NaN     翡翠明珠   1059500   94.12㎡  79.36㎡     3   
84129   6  93.0  247.0  二至三环    明德8英里   1087900  105.45㎡  83.23㎡     3   
84130   6  46.0  597.0   NaN     魏玛公馆    967200    53.8㎡  39.42㎡     1   
84131   6  93.0  247.0  二至三环     万国花园   1095000  136.74㎡     NaN     0   
84132   6  46.0  597.0   NaN   毛纺厂家属院    434700   63.02㎡  55.72㎡     0   

          建筑结构  ...                                               户型介绍  \
0       9405.0  ...                  

### 10. 配备电梯

In [29]:
total['配备电梯'] = total['配备电梯'].fillna('无')
# 创建一个映射字典
order_mapping = {
    '无': 0,
    '有': 1,
}

# 使用映射字典进行编码
total['配备电梯'] = total['配备电梯'].map(order_mapping)

print(total[["配备电梯"]])

       配备电梯
0         0
1         0
2         0
3         1
4         1
...     ...
84128     1
84129     1
84130     1
84131     0
84132     0

[84133 rows x 1 columns]


### 11. 别墅类型

In [30]:
total['别墅类型'] = total['别墅类型'].fillna('非别墅')
# 使用 pd.get_dummies进行独热编码
total = pd.get_dummies(total, columns=['别墅类型'])
# 输出结果
print(total)

       城市    区域     板块    环线     小区名称        价格     建筑面积    套内面积  房屋朝向  \
0       0  79.0  111.0  二至三环    人定湖西里   6564200    52.3㎡     NaN     0   
1       0  43.0  231.0  五至六环    龙跃苑四区   4174000  127.44㎡  123.7㎡     0   
2       0  97.0   54.0  五至六环      名都园  16310000  228.54㎡     NaN     0   
3       0  62.0  568.0  三至四环   保利海德公园   2834600    43.6㎡  29.39㎡     1   
4       0  62.0  226.0  三至四环  京投银泰琨御府   1954000   39.85㎡  29.94㎡     0   
...    ..   ...    ...   ...      ...       ...      ...     ...   ...   
84128   6  71.0  176.0   NaN     翡翠明珠   1059500   94.12㎡  79.36㎡     3   
84129   6  93.0  247.0  二至三环    明德8英里   1087900  105.45㎡  83.23㎡     3   
84130   6  46.0  597.0   NaN     魏玛公馆    967200    53.8㎡  39.42㎡     1   
84131   6  93.0  247.0  二至三环     万国花园   1095000  136.74㎡     NaN     0   
84132   6  46.0  597.0   NaN   毛纺厂家属院    434700   63.02㎡  55.72㎡     0   

          建筑结构  ...      年份  卧室  客厅 厨房 卫生间 别墅类型_双拼 别墅类型_叠拼 别墅类型_独栋 别墅类型_联排  \
0       9405.0  ...  2018.0   2  

### 12. 交易时间

In [31]:
# 先确保转换为 datetime 类型
total['交易时间'] = pd.to_datetime(total['交易时间'],errors = 'coerce')

# 计算数值化的年份
total['交易时间'] = total['交易时间'].dt.year + (total['交易时间'].dt.dayofyear / 365)

print(total[["交易时间"]])

              交易时间
0      2020.613699
1      2020.200000
2      2020.893151
3      2019.123288
4      2019.317808
...            ...
84128  2024.569863
84129  2024.572603
84130  2024.572603
84131  2024.572603
84132  2024.572603

[84133 rows x 1 columns]


### 13. 交易权属

In [32]:
# 使用 pd.get_dummies进行独热编码
total = pd.get_dummies(total, columns=['交易权属'])
# 输出结果
print(total)

       城市    区域     板块    环线     小区名称        价格     建筑面积    套内面积  房屋朝向  \
0       0  79.0  111.0  二至三环    人定湖西里   6564200    52.3㎡     NaN     0   
1       0  43.0  231.0  五至六环    龙跃苑四区   4174000  127.44㎡  123.7㎡     0   
2       0  97.0   54.0  五至六环      名都园  16310000  228.54㎡     NaN     0   
3       0  62.0  568.0  三至四环   保利海德公园   2834600    43.6㎡  29.39㎡     1   
4       0  62.0  226.0  三至四环  京投银泰琨御府   1954000   39.85㎡  29.94㎡     0   
...    ..   ...    ...   ...      ...       ...      ...     ...   ...   
84128   6  71.0  176.0   NaN     翡翠明珠   1059500   94.12㎡  79.36㎡     3   
84129   6  93.0  247.0  二至三环    明德8英里   1087900  105.45㎡  83.23㎡     3   
84130   6  46.0  597.0   NaN     魏玛公馆    967200    53.8㎡  39.42㎡     1   
84131   6  93.0  247.0  二至三环     万国花园   1095000  136.74㎡     NaN     0   
84132   6  46.0  597.0   NaN   毛纺厂家属院    434700   63.02㎡  55.72㎡     0   

          建筑结构  ...  交易权属_央产房  交易权属_定向安置房  交易权属_已购公房  交易权属_房改房 交易权属_拆迁还建房  \
0       9405.0  ...         0     

### 14. 上次交易  


In [33]:

# 先确保转换为 datetime 类型
total['上次交易'] = pd.to_datetime(total['上次交易'],errors = 'coerce')

# 计算数值化的年份
total['上次交易'] = total['上次交易'].dt.year + (total['上次交易'].dt.dayofyear / 365)


### 15. 房屋用途

In [34]:
# 进行频数编码
total['房屋用途'] = total['房屋用途'].map(total['房屋用途'].value_counts())
# 查看结果
print(total[['房屋用途']])

          房屋用途
0      79853.0
1      79853.0
2      79853.0
3       1178.0
4       1178.0
...        ...
84128  79853.0
84129  79853.0
84130  79853.0
84131  79853.0
84132  79853.0

[84133 rows x 1 columns]


### 16. 房屋年限 

In [35]:
total['房屋年限'] = total['房屋年限'].fillna('未知')

# 创建一个映射字典
order_mapping = {
    '满五年': 5,
    '满两年': 2,
    '未满两年': 1,
    '未知':3.5 
}

# 使用映射字典进行编码
total['房屋年限'] = total['房屋年限'].map(order_mapping)
print(total[["房屋年限"]])

       房屋年限
0       5.0
1       5.0
2       5.0
3       5.0
4       5.0
...     ...
84128   5.0
84129   5.0
84130   5.0
84131   5.0
84132   5.0

[84133 rows x 1 columns]


### 17. 产权所属

In [36]:
# 创建一个映射字典
order_mapping = {
    '共有': 0,
    '非共有': 1,
}

# 使用映射字典进行编码
total['产权所属'] = total['产权所属'].map(order_mapping)

print(total[["产权所属"]])

       产权所属
0         1
1         1
2         1
3         1
4         1
...     ...
84128     0
84129     1
84130     1
84131     1
84132     1

[84133 rows x 1 columns]


### 18. 房屋优势

In [37]:
total.drop(columns=['房屋优势'], inplace=True)
print(total)

       城市    区域     板块    环线     小区名称        价格     建筑面积    套内面积  房屋朝向  \
0       0  79.0  111.0  二至三环    人定湖西里   6564200    52.3㎡     NaN     0   
1       0  43.0  231.0  五至六环    龙跃苑四区   4174000  127.44㎡  123.7㎡     0   
2       0  97.0   54.0  五至六环      名都园  16310000  228.54㎡     NaN     0   
3       0  62.0  568.0  三至四环   保利海德公园   2834600    43.6㎡  29.39㎡     1   
4       0  62.0  226.0  三至四环  京投银泰琨御府   1954000   39.85㎡  29.94㎡     0   
...    ..   ...    ...   ...      ...       ...      ...     ...   ...   
84128   6  71.0  176.0   NaN     翡翠明珠   1059500   94.12㎡  79.36㎡     3   
84129   6  93.0  247.0  二至三环    明德8英里   1087900  105.45㎡  83.23㎡     3   
84130   6  46.0  597.0   NaN     魏玛公馆    967200    53.8㎡  39.42㎡     1   
84131   6  93.0  247.0  二至三环     万国花园   1095000  136.74㎡     NaN     0   
84132   6  46.0  597.0   NaN   毛纺厂家属院    434700   63.02㎡  55.72㎡     0   

          建筑结构  ...  交易权属_央产房  交易权属_定向安置房  交易权属_已购公房  交易权属_房改房  交易权属_拆迁还建房  \
0       9405.0  ...         0    

### 19&20&21&22  


In [38]:
def label(column_name):
    # 非空值为1，空值为0
    total[column_name] = total[column_name].isna().astype(int)
    return total
for i in ['核心卖点','户型介绍','周边配套','交通出行']:
    label(i)
print(total)

       城市    区域     板块    环线     小区名称        价格     建筑面积    套内面积  房屋朝向  \
0       0  79.0  111.0  二至三环    人定湖西里   6564200    52.3㎡     NaN     0   
1       0  43.0  231.0  五至六环    龙跃苑四区   4174000  127.44㎡  123.7㎡     0   
2       0  97.0   54.0  五至六环      名都园  16310000  228.54㎡     NaN     0   
3       0  62.0  568.0  三至四环   保利海德公园   2834600    43.6㎡  29.39㎡     1   
4       0  62.0  226.0  三至四环  京投银泰琨御府   1954000   39.85㎡  29.94㎡     0   
...    ..   ...    ...   ...      ...       ...      ...     ...   ...   
84128   6  71.0  176.0   NaN     翡翠明珠   1059500   94.12㎡  79.36㎡     3   
84129   6  93.0  247.0  二至三环    明德8英里   1087900  105.45㎡  83.23㎡     3   
84130   6  46.0  597.0   NaN     魏玛公馆    967200    53.8㎡  39.42㎡     1   
84131   6  93.0  247.0  二至三环     万国花园   1095000  136.74㎡     NaN     0   
84132   6  46.0  597.0   NaN   毛纺厂家属院    434700   63.02㎡  55.72㎡     0   

          建筑结构  ...  交易权属_央产房  交易权属_定向安置房  交易权属_已购公房  交易权属_房改房  交易权属_拆迁还建房  \
0       9405.0  ...         0    

In [39]:
community = pd.read_csv('/home/mw/project/ruc_Class25Q1_community.csv')  

## 合并与划分

In [40]:
# 合并小区数据到房价数据
merged = total.merge(community, on='小区名称', how='left')
print(merged)

       城市    区域     板块    环线     小区名称        价格     建筑面积    套内面积  房屋朝向  \
0       0  79.0  111.0  二至三环    人定湖西里   6564200    52.3㎡     NaN     0   
1       0  43.0  231.0  五至六环    龙跃苑四区   4174000  127.44㎡  123.7㎡     0   
2       0  97.0   54.0  五至六环      名都园  16310000  228.54㎡     NaN     0   
3       0  62.0  568.0  三至四环   保利海德公园   2834600    43.6㎡  29.39㎡     1   
4       0  62.0  226.0  三至四环  京投银泰琨御府   1954000   39.85㎡  29.94㎡     0   
...    ..   ...    ...   ...      ...       ...      ...     ...   ...   
91608   6  71.0  176.0   NaN     翡翠明珠   1059500   94.12㎡  79.36㎡     3   
91609   6  93.0  247.0  二至三环    明德8英里   1087900  105.45㎡  83.23㎡     3   
91610   6  46.0  597.0   NaN     魏玛公馆    967200    53.8㎡  39.42㎡     1   
91611   6  93.0  247.0  二至三环     万国花园   1095000  136.74㎡     NaN     0   
91612   6  46.0  597.0   NaN   毛纺厂家属院    434700   63.02㎡  55.72㎡     0   

          建筑结构  ...  绿 化 率  容 积 率   物 业 费     供水        供暖     供电    燃气费  \
0       9405.0  ...     30   3.00  

In [41]:
merged['环线'] = merged['环线'].fillna(merged['环线位置'])
merged.drop(columns = ['环线位置'], inplace = True)
merged.drop(columns = ['小区名称'], inplace = True)

In [42]:
merged["供暖"].value_counts()


集中供暖            36415
自采暖             16307
集中供暖/自采暖         9247
集中供暖/自采暖/无供暖       70
自采暖/无供暖            65
无供暖                 4
Name: 供暖, dtype: int64

In [43]:
most_frequent_value = merged['供暖'].mode()[0]
merged['供暖'] = merged['供暖'].fillna(most_frequent_value)
# 供暖方式类别
heating_types = ['集中供暖', '自采暖', '无供暖']

# 创建独热编码列
for heating in heating_types:
    merged[heating] = merged['供暖'].apply(lambda x: 1 if heating in x else 0)

merged.drop(columns = ['供暖'], inplace = True)
print(merged)


       城市    区域     板块    环线        价格     建筑面积    套内面积  房屋朝向     建筑结构  装修情况  \
0       0  79.0  111.0  二至三环   6564200    52.3㎡     NaN     0   9405.0     3   
1       0  43.0  231.0  五至六环   4174000  127.44㎡  123.7㎡     0   9405.0     3   
2       0  97.0   54.0  五至六环  16310000  228.54㎡     NaN     0   9405.0     3   
3       0  62.0  568.0  三至四环   2834600    43.6㎡  29.39㎡     1  65198.0     3   
4       0  62.0  226.0  三至四环   1954000   39.85㎡  29.94㎡     0  65198.0     3   
...    ..   ...    ...   ...       ...      ...     ...   ...      ...   ...   
91608   6  71.0  176.0   NaN   1059500   94.12㎡  79.36㎡     3   2462.0     3   
91609   6  93.0  247.0  二至三环   1087900  105.45㎡  83.23㎡     3  65198.0     3   
91610   6  46.0  597.0   NaN    967200    53.8㎡  39.42㎡     1  65198.0     3   
91611   6  93.0  247.0  二至三环   1095000  136.74㎡     NaN     0   9405.0     3   
91612   6  46.0  597.0   NaN    434700   63.02㎡  55.72㎡     0   9405.0     0   

       ...   物 业 费     供水     供电    燃气费

In [44]:
merged["供水"].value_counts()

民水       50858
商水/民水    32913
商水         450
Name: 供水, dtype: int64

In [45]:
most_frequent_value = merged['供水'].mode()[0]
merged['供水'] = merged['供水'].fillna(most_frequent_value)
waterSupply_types = ['民水','商水']

# 创建独热编码列
for waterSupply in waterSupply_types:
    merged[waterSupply] = merged['供水'].apply(lambda x: 1 if waterSupply in x else 0)

merged.drop(columns = ['供水'], inplace = True)
print(merged)

       城市    区域     板块    环线        价格     建筑面积    套内面积  房屋朝向     建筑结构  装修情况  \
0       0  79.0  111.0  二至三环   6564200    52.3㎡     NaN     0   9405.0     3   
1       0  43.0  231.0  五至六环   4174000  127.44㎡  123.7㎡     0   9405.0     3   
2       0  97.0   54.0  五至六环  16310000  228.54㎡     NaN     0   9405.0     3   
3       0  62.0  568.0  三至四环   2834600    43.6㎡  29.39㎡     1  65198.0     3   
4       0  62.0  226.0  三至四环   1954000   39.85㎡  29.94㎡     0  65198.0     3   
...    ..   ...    ...   ...       ...      ...     ...   ...      ...   ...   
91608   6  71.0  176.0   NaN   1059500   94.12㎡  79.36㎡     3   2462.0     3   
91609   6  93.0  247.0  二至三环   1087900  105.45㎡  83.23㎡     3  65198.0     3   
91610   6  46.0  597.0   NaN    967200    53.8㎡  39.42㎡     1  65198.0     3   
91611   6  93.0  247.0  二至三环   1095000  136.74㎡     NaN     0   9405.0     3   
91612   6  46.0  597.0   NaN    434700   63.02㎡  55.72㎡     0   9405.0     0   

       ...     供电    燃气费   供热费     停车位 

In [46]:
merged["供电"].value_counts()

民电       51017
商电/民电    33359
商电         347
Name: 供电, dtype: int64

In [47]:
most_frequent_value = merged['供电'].mode()[0]
merged['供电'] = merged['供电'].fillna(most_frequent_value)
elecSupply_types = ['民电','商电']

# 创建独热编码列
for elecSupply in elecSupply_types:
    merged[elecSupply] = merged['供电'].apply(lambda x: 1 if elecSupply in x else 0)

merged.drop(columns = ['供电'], inplace = True)
print(merged)

       城市    区域     板块    环线        价格     建筑面积    套内面积  房屋朝向     建筑结构  装修情况  \
0       0  79.0  111.0  二至三环   6564200    52.3㎡     NaN     0   9405.0     3   
1       0  43.0  231.0  五至六环   4174000  127.44㎡  123.7㎡     0   9405.0     3   
2       0  97.0   54.0  五至六环  16310000  228.54㎡     NaN     0   9405.0     3   
3       0  62.0  568.0  三至四环   2834600    43.6㎡  29.39㎡     1  65198.0     3   
4       0  62.0  226.0  三至四环   1954000   39.85㎡  29.94㎡     0  65198.0     3   
...    ..   ...    ...   ...       ...      ...     ...   ...      ...   ...   
91608   6  71.0  176.0   NaN   1059500   94.12㎡  79.36㎡     3   2462.0     3   
91609   6  93.0  247.0  二至三环   1087900  105.45㎡  83.23㎡     3  65198.0     3   
91610   6  46.0  597.0   NaN    967200    53.8㎡  39.42㎡     1  65198.0     3   
91611   6  93.0  247.0  二至三环   1095000  136.74㎡     NaN     0   9405.0     3   
91612   6  46.0  597.0   NaN    434700   63.02㎡  55.72㎡     0   9405.0     0   

       ...   供热费     停车位  停车费用  集中供暖  自

### 处理环线数据

In [48]:
mapping = {
    "一环内": 1,
    "内环内": 2,
    "二环内": 3,
    "一至二环": 1.5,
    "二至三环": 2.5,
    "三至四环": 3.5,
    "四至五环": 4.5,
    "五至六环": 5.5,
    "六环外": 6,
    "三环外": 4.5,
    "四环外": 5,
    "外环外": 7,
    "内环至中环": 3,
    "中环至外环": 5,
    "内环至外环": 4
}

merged["环线"] = merged["环线"].replace(mapping)

In [49]:
merged['绿 化 率'] = pd.to_numeric(merged['绿 化 率'], errors='coerce')
merged.drop(columns = ["抵押信息"], inplace = True)

In [50]:
# 确保所有数据都转换为字符串类型后去除平方米单位
merged['建筑面积'] = merged['建筑面积'].astype(str).str.replace('㎡', '', regex=True).astype(float)
merged['套内面积'] = merged['套内面积'].astype(str).str.replace('㎡', '', regex=True)
merged['套内面积'] = pd.to_numeric(merged['套内面积'], errors='coerce')  # 转换为数值，无法转换的变为 NaN


In [51]:
merged.to_csv('/home/mw/project/ruc_Class25Q1_merged.csv', index=False)

In [5]:
merged = pd.read_csv('/home/mw/project/ruc_Class25Q1_merged.csv')
merged["价格"]=np.log1p(merged["价格"])

## 利用训练集进行缺失值填充 and 处理一些刚刚merge进来的变量

In [6]:
# 选择自变量 X（去掉目标变量 '房产价格'）和因变量 y（房产价格）
X = merged.drop(columns=['价格'])  
y = merged['价格']  

# 按 80% 训练集，20% 测试集划分，并设置 random_state=111
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

# 输出数据集大小
print(f"训练集样本数: {X_train.shape[0]}")
print(f"测试集样本数: {X_test.shape[0]}")

训练集样本数: 73290
测试集样本数: 18323


In [7]:
# 按建筑面积降序排序
X_train = X_train.sort_values(by='建筑面积', ascending=False).reset_index(drop=True)

# 使用建筑面积作为参考，但只填充套内面积
temp_X_train = X_train[['建筑面积', '套内面积']]
imputer = KNNImputer(n_neighbors=10)
X_train['套内面积'] = imputer.fit_transform(temp_X_train)[:, 1]  # 仅填充套内面积

# 确保填充后的套内面积不会超过建筑面积
X_train['套内面积'] = np.minimum(X_train['套内面积'], X_train['建筑面积'])

# 计算得房率（套内面积 / 建筑面积）
X_train['得房率'] = X_train['套内面积'] / X_train['建筑面积']

In [8]:
for col in X_train.iloc[:, 0:].columns:  # 选取从第4列（索引3）往后的所有列
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')  # 转换为数值，错误变 NaN
    if X_train[col].notna().sum() > 0:  # 仅在列中有有效数字时才计算众数
        mode_value = X_train[col].mode()[0]  # 获取众数
        X_train[col].fillna(mode_value, inplace=True)  # 用众数填充 NaN

print(X_train)

       城市    区域     板块   环线      建筑面积    套内面积  房屋朝向     建筑结构  装修情况    梯户比例  \
0       6  66.0  307.0  2.5  10337.00  667.78     0  65198.0     0  0.3333   
1       5  48.0  438.0  2.5   5108.00  667.78     0  65198.0     2  0.3333   
2       5  82.0  234.0  2.5   1389.39  667.78     0   2462.0     0  0.3333   
3       6  93.0  776.0  2.5   1250.00  667.78     0  65198.0     0  0.3333   
4       6  93.0  776.0  2.5   1250.00  667.78     0  65198.0     0  1.0000   
...    ..   ...    ...  ...       ...     ...   ...      ...   ...     ...   
73285   6  93.0  250.0  2.5     10.00   10.00     0  65198.0     0  0.5000   
73286   6  46.0  238.0  4.5     10.00   10.00     0  65198.0     0  0.5000   
73287   6  93.0  733.0  2.5     10.00   10.00     3  65198.0     0  0.5000   
73288   6  66.0  596.0  2.5     10.00   10.00     0  65198.0     0  0.5000   
73289   6  66.0  596.0  2.5     10.00   10.00     0  65198.0     0  0.5000   

       ...     停车位   停车费用  集中供暖  自采暖  无供暖  民水  商水  民电  商电      

In [9]:
# 计算得房率（套内面积 / 建筑面积）
X_test['得房率'] = X_test['套内面积'] / X_test['建筑面积']

In [10]:
# **步骤 1: 对分类变量进行编码（Label Encoding）**
encoders = {}  # 存储编码器，方便后续转换回原始值

for col in X_train.columns:
    le = LabelEncoder()
    combined_data = pd.concat([X_train[col], X_test[col]], axis=0)  # 训练 & 测试集一起编码
    combined_data = combined_data.astype(str).fillna("MISSING")  # 处理 NaN
    le.fit(combined_data)
    
    X_train[col] = le.transform(X_train[col].astype(str).fillna("MISSING"))
    X_test[col] = le.transform(X_test[col].astype(str).fillna("MISSING"))
    
    encoders[col] = le  # 存储编码器

# **步骤 2: 进行 KNN 填充**
knn_imputer = KNNImputer(n_neighbors=3, weights="uniform")  # k=3

# 只对数值数据进行填充
train_filled = knn_imputer.fit_transform(X_train)
test_filled = knn_imputer.transform(X_test)

# **步骤 3: 还原编码回原始类别**
train_filled_df = pd.DataFrame(train_filled, columns=X_train.columns)
test_filled_df = pd.DataFrame(test_filled, columns=X_test.columns)

for col in train_filled_df.columns:
    le = encoders[col]  # 获取之前保存的 LabelEncoder
    train_filled_df[col] = le.inverse_transform(train_filled_df[col].round().astype(int))
    test_filled_df[col] = le.inverse_transform(test_filled_df[col].round().astype(int))

# **最终输出**
print("KNN 填充后的训练集：\n", train_filled_df)
print("\nKNN 填充后的测试集：\n", test_filled_df)

KNN 填充后的训练集：
       城市    区域     板块   环线     建筑面积    套内面积 房屋朝向     建筑结构 装修情况    梯户比例  ...  \
0      6  66.0  307.0  2.5  10337.0  667.78    0  65198.0    0  0.3333  ...   
1      5  48.0  438.0  2.5   5108.0  667.78    0  65198.0    2  0.3333  ...   
2      5  82.0  234.0  2.5  1389.39  667.78    0   2462.0    0  0.3333  ...   
3      6  93.0  776.0  2.5   1250.0  667.78    0  65198.0    0  0.3333  ...   
4      6  93.0  776.0  2.5   1250.0  667.78    0  65198.0    0     1.0  ...   
...   ..   ...    ...  ...      ...     ...  ...      ...  ...     ...  ...   
73285  6  93.0  250.0  2.5     10.0    10.0    0  65198.0    0     0.5  ...   
73286  6  46.0  238.0  4.5     10.0    10.0    0  65198.0    0     0.5  ...   
73287  6  93.0  733.0  2.5     10.0    10.0    3  65198.0    0     0.5  ...   
73288  6  66.0  596.0  2.5     10.0    10.0    0  65198.0    0     0.5  ...   
73289  6  66.0  596.0  2.5     10.0    10.0    0  65198.0    0     0.5  ...   

          停车位   停车费用 集中供暖 自采暖 无供暖 民水 

In [11]:
for col in X_train.columns:
    # 强制转换为数值类型，错误值变 NaN
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')

    # 只有在列中有非 NaN 值时才计算众数
    if X_train[col].notna().sum() > 0:
        if not X_train[col].mode().empty:  # 确保 mode() 不为空
            mode_value = X_train[col].mode()[0]  # 获取众数
            X_train[col].fillna(mode_value, inplace=True)  # 用众数填充 NaN
        else:
            X_train[col].fillna(0, inplace=True)  # 如果 mode() 为空，则填充 0

# 再次检查是否还有 NaN
print("剩余 NaN 数量:\n", X_train.isna().sum().sum())

# **输出结果**
print("训练集：\n", X_train)
print("\n测试集：\n", X_test)

剩余 NaN 数量:
 0
训练集：
        城市  区域   板块  环线  建筑面积   套内面积  房屋朝向  建筑结构  装修情况  梯户比例  ...  停车位  停车费用  \
0       6  56  187   3   383  11634     0     4     0   112  ...  212    34   
1       5  37  307   3  9457  11634     0     4     2   112  ...  439   165   
2       5  72  124   3  3470  11634     0     1     0   112  ...  498    45   
3       6  83  611   3  2342  11634     0     4     0   112  ...  498    45   
4       6  83  611   3  2342  11634     0     4     0   132  ...  498    45   
...    ..  ..  ...  ..   ...    ...   ...   ...   ...   ...  ...  ...   ...   
73285   6  83  139   3     0      1     0     4     0   121  ...   29    45   
73286   6  35  127   7     0      1     0     4     0   121  ...  354    45   
73287   6  83  573   3     0      1     3     4     0   121  ...  269    45   
73288   6  56  448   3     0      1     0     4     0   121  ...   93   113   
73289   6  56  448   3     0      1     0     4     0   121  ...   93   113   

       集中供暖  自采暖  无供暖  民水  商水  

In [12]:
X_train.to_csv('/home/mw/project/ruc_Class25Q1_xtrain.csv', index=False)
y_train.to_csv('/home/mw/project/ruc_Class25Q1_ytrain.csv', index=False)
X_test.to_csv('/home/mw/project/ruc_Class25Q1_xtest.csv', index=False)
y_test.to_csv('/home/mw/project/ruc_Class25Q1_ytest.csv', index=False)

In [13]:
X_train = pd.read_csv('/home/mw/project/ruc_Class25Q1_xtrain.csv')
y_train = pd.read_csv('/home/mw/project/ruc_Class25Q1_ytrain.csv')  
X_test = pd.read_csv('/home/mw/project/ruc_Class25Q1_xtest.csv')  
y_test = pd.read_csv('/home/mw/project/ruc_Class25Q1_ytest.csv')  

In [14]:
print(X_train.columns)

Index(['城市', '区域', '板块', '环线', '建筑面积', '套内面积', '房屋朝向', '建筑结构', '装修情况', '梯户比例',
       '配备电梯', '交易时间', '上次交易', '房屋用途', '房屋年限', '产权所属', '核心卖点', '户型介绍', '周边配套',
       '交通出行', 'lon', 'lat', '年份', '卧室', '客厅', '厨房', '卫生间', '别墅类型_双拼',
       '别墅类型_叠拼', '别墅类型_独栋', '别墅类型_联排', '别墅类型_非别墅', '交易权属_一类经济适用房',
       '交易权属_二类经济适用房', '交易权属_使用权', '交易权属_动迁安置房', '交易权属_售后公房', '交易权属_商品房',
       '交易权属_央产房', '交易权属_定向安置房', '交易权属_已购公房', '交易权属_房改房', '交易权属_拆迁还建房',
       '交易权属_私产', '交易权属_经济适用房', '交易权属_自住型商品房', '交易权属_限价商品房', '交易权属_集资房',
       '房屋总数', '楼栋总数', '绿 化 率', '容 积 率', '物 业 费', '燃气费', '供热费', '停车位', '停车费用',
       '集中供暖', '自采暖', '无供暖', '民水', '商水', '民电', '商电', '得房率'],
      dtype='object')


In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 计算 X_train 的所有变量与 y_train（价格）的相关性
correlation_with_target = X_train.join(y_train).corr()["价格"].abs().sort_values(ascending=False)

# 选择最相关的变量（如相关性 > 0.3）
selected_features = correlation_with_target[correlation_with_target > 0.003].index.tolist()
selected_features.remove("价格")  # 移除目标变量

print("选择的变量:", selected_features)

# 只用最相关的变量进行回归
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# 训练回归模型
model = LinearRegression()
model.fit(X_train_selected, y_train)

# 进行预测
y_pred = model.predict(X_test_selected)

# 评估模型
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae}, RMSE: {rmse}")

predictiontest = pd.read_csv('/home/mw/project/ruc_Class25Q1_predictiontest.csv')  
# 只保留模型需要的特征
predictiontest_selected = predictiontest[selected_features]

# 确保数据格式正确
predictiontest_selected = predictiontest_selected.astype(float)

# 进行预测
y_pred = model.predict(predictiontest_selected)

# 确保 y_pred 是 1D 数组
y_pred = y_pred.ravel()  
y_pred = np.abs(y_pred)

# 创建最终的 DataFrame（第一列是序号，第二列是预测结果）
df_result = pd.DataFrame({
    "ID": predictiontest.iloc[:, 0],  # 取回原来的序号
    "Price": y_pred
})
# 生成带模型名称的文件名
filename = "/home/mw/project/predicted_prices_bla.csv"

# 保存 CSV
df_result.to_csv(filename, index=False, encoding="utf-8-sig")



选择的变量: ['燃气费', '交易权属_房改房', '城市', '房屋朝向', 'lon', '交易权属_一类经济适用房', '周边配套', '别墅类型_叠拼', '卫生间', '建筑结构', '产权所属', '客厅', '房屋年限', '自采暖', 'lat', '交易权属_央产房', '交易权属_售后公房', '梯户比例', '绿 化 率', '环线', '交易权属_使用权']
MAE: 0.7127610911771923, RMSE: 0.9038522272259434


In [16]:
# 确保 y_train 是一个 DataFrame
if isinstance(y_train, pd.Series):
    y_train = y_train.to_frame()

# 计算 X_train 与 y_train 之间的相关性
correlation_matrix = X_train.corrwith(y_train.iloc[:, 0]).abs()  # 取绝对值，关注强度

# 设定相关系数阈值（例如：0.02 以上认为相关）
threshold = 0.003
selected_features = correlation_matrix[correlation_matrix > threshold].index.tolist()

# 输出相关系数排名
print("相关系数排名：")
print(correlation_matrix.sort_values(ascending=False))

# 输出被选择的变量
print("\n适合线性回归的变量（相关系数 > 0.003）：")
print(selected_features)


相关系数排名：
燃气费             0.007501
交易权属_房改房        0.007401
城市              0.007031
房屋朝向            0.006917
lon             0.006801
                  ...   
房屋总数            0.000236
年份              0.000216
集中供暖            0.000130
交易权属_二类经济适用房    0.000122
商水              0.000042
Length: 65, dtype: float64

适合线性回归的变量（相关系数 > 0.003）：
['城市', '环线', '房屋朝向', '建筑结构', '梯户比例', '房屋年限', '产权所属', '周边配套', 'lon', 'lat', '客厅', '卫生间', '别墅类型_叠拼', '交易权属_一类经济适用房', '交易权属_使用权', '交易权属_售后公房', '交易权属_央产房', '交易权属_房改房', '绿 化 率', '燃气费', '自采暖']


In [17]:
import numpy as np

# 检查 NaN 值
print("X_train 是否有 NaN:", X_train.isna().sum().sum())  
print("y_train 是否有 NaN:", y_train.isna().sum().sum())

# 检查 Inf 值
print("X_train 是否有 Inf:", np.isinf(X_train).sum().sum())
print("y_train 是否有 Inf:", np.isinf(y_train).sum().sum())


X_train 是否有 NaN: 0
y_train 是否有 NaN: 0
X_train 是否有 Inf: 0
y_train 是否有 Inf: 0


In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 训练模型
lr = LinearRegression()
lr.fit(X_train, y_train)

# 预测
y_pred = lr.predict(X_test)

# 评估
print("Linear Regression:")
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


Linear Regression:
MSE: 0.8232235441243069
R² Score: -0.021278942306139248


In [33]:
# 1. 检查并填充 NaN
X_train.fillna(X_train.median(), inplace=True)
y_train.fillna(y_train.median(), inplace=True)

# 2. 确保所有变量都是数值型
X_train = pd.get_dummies(X_train, drop_first=True)

# 3. 重新检查数据类型 & 确保没有 NaN
assert not X_train.isnull().values.any(), "X_train 仍然包含 NaN!"
assert not y_train.isnull().values.any(), "y_train 仍然包含 NaN!"
assert X_train.shape[0] == y_train.shape[0], f"X_train ({X_train.shape[0]}) 和 y_train ({y_train.shape[0]}) 维度不匹配!"

# 4. 进行交叉验证
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

kf = KFold(n_splits=6, shuffle=True, random_state=42)
model = LinearRegression()

mae_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')

print("Cross-Validation MAE:", -mae_scores.mean())  # 取负数

Cross-Validation MAE: 0.7053670499892178


In [27]:
from sklearn.linear_model import LassoCV

# 交叉验证选择最佳 α（L1 正则化强度）
lasso = LassoCV(alphas=[0.001, 0.01, 0.1, 1, 10], cv=5)
lasso.fit(X_train, y_train)

# 预测
y_pred = lasso.predict(X_test)

# 评估
print("\nLasso Regression:")
print("Best alpha:", lasso.alpha_)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

# 选择的变量
selected_features = X_train.columns[lasso.coef_ != 0]
print("Lasso 选择的变量:", selected_features)


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Lasso Regression:
Best alpha: 10.0
MSE: 0.8062513627515463
R² Score: -0.00022350534164017333
Lasso 选择的变量: Index(['建筑面积', '得房率'], dtype='object')


In [31]:
# 1. 检查并填充 NaN
X_train.fillna(X_train.median(), inplace=True)
y_train.fillna(y_train.median(), inplace=True)

# 2. 确保所有变量都是数值型
X_train = pd.get_dummies(X_train, drop_first=True)

# 3. 重新检查数据类型 & 确保没有 NaN
assert not X_train.isnull().values.any(), "X_train 仍然包含 NaN!"
assert not y_train.isnull().values.any(), "y_train 仍然包含 NaN!"
assert X_train.shape[0] == y_train.shape[0], f"X_train ({X_train.shape[0]}) 和 y_train ({y_train.shape[0]}) 维度不匹配!"

# 4. 进行交叉验证
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

kf = KFold(n_splits=6, shuffle=True, random_state=42)
model = LassoCV()

mae_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')

print("Cross-Validation MAE:", -mae_scores.mean())  # 取负数

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expec

Cross-Validation MAE: 0.7048252332594865


In [20]:
from sklearn.linear_model import RidgeCV

# 交叉验证选择最佳 α（L2 正则化强度）
ridge = RidgeCV(alphas=[0.001, 0.01, 0.1, 1, 10], cv=5)
ridge.fit(X_train, y_train)

# 预测
y_pred = ridge.predict(X_test)

# 评估
print("\nRidge Regression:")
print("Best alpha:", ridge.alpha_)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))



Ridge Regression:
Best alpha: 10.0
MSE: 0.8231135567642033
R² Score: -0.021142493615385227


In [21]:
# 1. 检查并填充 NaN
X_train.fillna(X_train.median(), inplace=True)
y_train.fillna(y_train.median(), inplace=True)

# 2. 确保所有变量都是数值型
X_train = pd.get_dummies(X_train, drop_first=True)

# 3. 重新检查数据类型 & 确保没有 NaN
assert not X_train.isnull().values.any(), "X_train 仍然包含 NaN!"
assert not y_train.isnull().values.any(), "y_train 仍然包含 NaN!"
assert X_train.shape[0] == y_train.shape[0], f"X_train ({X_train.shape[0]}) 和 y_train ({y_train.shape[0]}) 维度不匹配!"

# 4. 进行交叉验证
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

kf = KFold(n_splits=6, shuffle=True, random_state=42)
model = Ridge()

mae_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')

print("Cross-Validation MAE:", -mae_scores.mean())  # 取负数

Cross-Validation MAE: 0.7053619293298795


In [22]:
from sklearn.linear_model import ElasticNetCV

# 交叉验证选择最佳 α 和 L1_ratio
elastic_net = ElasticNetCV(alphas=[0.001, 0.01, 0.1, 1, 10], l1_ratio=[0.1, 0.5, 0.9], cv=5)
elastic_net.fit(X_train, y_train)

# 预测
y_pred = elastic_net.predict(X_test)

# 评估
print("\nElastic Net Regression:")
print("Best alpha:", elastic_net.alpha_)
print("Best L1 ratio:", elastic_net.l1_ratio_)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Elastic Net Regression:
Best alpha: 10.0
Best L1 ratio: 0.9
MSE: 0.8063045242725644
R² Score: -0.0002894567376421353


In [23]:
# 1. 检查并填充 NaN
X_train.fillna(X_train.median(), inplace=True)
y_train.fillna(y_train.median(), inplace=True)

# 2. 确保所有变量都是数值型
X_train = pd.get_dummies(X_train, drop_first=True)

# 3. 重新检查数据类型 & 确保没有 NaN
assert not X_train.isnull().values.any(), "X_train 仍然包含 NaN!"
assert not y_train.isnull().values.any(), "y_train 仍然包含 NaN!"
assert X_train.shape[0] == y_train.shape[0], f"X_train ({X_train.shape[0]}) 和 y_train ({y_train.shape[0]}) 维度不匹配!"

# 4. 进行交叉验证
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge

kf = KFold(n_splits=6, shuffle=True, random_state=42)
model = ElasticNetCV()

mae_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')

print("Cross-Validation MAE:", -mae_scores.mean())  # 取负数


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:1572: DataConversionWarning: A column-vector y was passed when a 1d array was expec

Cross-Validation MAE: 0.704825233224002


In [24]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 训练 OLS、LASSO、Ridge 模型
models = {
    "OLS": LinearRegression(),
    "LASSO": LassoCV(alphas=[0.001, 0.01, 0.1, 1, 10], cv=5),
    "Ridge": RidgeCV(alphas=[0.001, 0.01, 0.1, 1, 10], cv=5),
    "elastic_net": ElasticNetCV(alphas=[0.001, 0.01, 0.1, 1, 10], l1_ratio=[0.1, 0.5, 0.9], cv=5)
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    
    # 计算 in-sample (训练集) 和 out-of-sample (测试集) 误差
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    mae_train = mean_absolute_error(y_train, y_train_pred)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    mae_test = mean_absolute_error(y_test, y_test_pred)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

# 使用 6 折交叉验证计算 MAE
    cv_mae = -cross_val_score(model, X, y, cv=6, scoring="neg_mean_absolute_error").mean()

    results[name] = {
        "MAE (Train)": mae_train,# 确保 y_train 是一个 DataFrame
        "RMSE (Train)": rmse_train,
        "MAE (Test)": mae_test,
        "RMSE (Test)": rmse_test,
        "Cross-Validation MAE": cv_mae
    }

# 选取最佳模型（假设选取 MAE 最小的）
best_model_name = min(results, key=lambda k: results[k]["MAE (Test)"])
best_model = models[best_model_name]

# 输出结果
merged_results = pd.DataFrame(results).T
print("\n模型评估结果:")
print(merged_results)
print(f"\n最佳模型: {best_model_name}")

# 输出去除异常值后的预测数量
print(f"\n去除异常值后，样本总数量: {merged.shape[0]}")

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
6 fits failed out of a total of 6.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_base.py", line 663, in fit
    X, y, accept_sparse=accept_sparse, y_numeric=True, multi_output=True
  File "/opt/conda/lib/python3.7/site-packages/sklearn/base.py", line 576, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/con


模型评估结果:
             MAE (Train)  RMSE (Train)  MAE (Test)  RMSE (Test)  \
OLS             0.704588      0.894191    0.716826     0.907317   
LASSO           0.704814      0.894501    0.708066     0.897915   
Ridge           0.704584      0.894192    0.716789     0.907256   
elastic_net     0.704814      0.894501    0.708091     0.897945   

             Cross-Validation MAE  
OLS                           NaN  
LASSO                         NaN  
Ridge                         NaN  
elastic_net                   NaN  

最佳模型: LASSO

去除异常值后，样本总数量: 91613


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
6 fits failed out of a total of 6.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py", line 1561, in fit
    X, y, validate_separately=(check_X_params, check_y_params)
  File "/opt/conda/lib/python3.7/site-packages/sklearn/base.py", line 573, in _validate_data
    X = check_array(X, **check_X_params)
  File "/opt/

# 用模型预测结果

In [34]:
predictiontest = pd.read_csv('/home/mw/project/ruc_Class25Q1_predictiontest.csv')  

In [36]:
predictiontest_fixed = predictiontest.iloc[:, 1:]  # 去掉第一列（假设第一列是序号）

# 定义模型名称
model_names = ["LinearRegression", "Lasso", "Ridge", "ElasticNet"]

# 遍历每个模型进行预测并保存结果
for model, name in zip([lr, lasso, ridge, elastic_net], model_names):
    y_pred = model.predict(predictiontest_fixed)

    # 确保 y_pred 是 1D 数组
    y_pred = y_pred.ravel() 
    y_pred=np.expm1(y_pred) 

    # 创建最终的 DataFrame（第一列是序号，第二列是预测结果）
    df_result = pd.DataFrame({
        "ID": predictiontest.iloc[:, 0],  # 取回原来的序号
        "Price": y_pred
    })
    # 生成带模型名称的文件名
    filename = f"/home/mw/project/predicted_prices_{name}.csv"

    # 保存 CSV
    df_result.to_csv(filename, index=False, encoding="utf-8-sig")

    

In [72]:
y_pred1 = lr.predict(predictiontest)
y_pred2 = lasso.predict(predictiontest)
y_pred3 = ridge.predict(predictiontest)
y_pred4 = elastic_net.predict(predictiontest)
y_pred1.to_csv('/home/mw/project/ruc_Class25Q1_OLS.csv', index=False)
y_pred2.to_csv('/home/mw/project/ruc_Class25Q1_Lasso.csv', index=False)
y_pred3.to_csv('/home/mw/project/ruc_Class25Q1_Ridge.csv', index=False)
y_pred4.to_csv('/home/mw/project/ruc_Class25Q1_Elastic_net.csv', index=False)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- ID
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: X has 66 features, but LinearRegression is expecting 65 features as input.